In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

#docstrings of the different self-made classes within the self-made module
#cdoc = calc.__doc__
#ddoc = det.__doc__
#rdoc = rf.__doc__

In [2]:
day = 150926
detector_name = 'n5'
data_type = 'ctime'
year = int('20' + str(day)[0:2])

#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))

#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

#read the measurement data
ctime_data = rf.ctime(detector_name, day)
echan = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

total_rate = np.sum(echan_rate[1:-2], axis = 0)

echan = 1 #define which energy-channels one wants to look at
counts = echan_rate[echan]
#counts = total_rate

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector_name, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#calculate the sun data
sun_data = calc.sun_ang_bin(detector_name, day, bin_time_mid, data_type)
sun_ang_bin = sun_data[0]
sun_ang_bin = calc.ang_eff(sun_ang_bin, echan)[0]
sun_rad = sun_data[2]
sun_ra = sun_rad[:,0]
sun_dec = sun_rad[:,1]
sun_occ = calc.src_occultation_bin(day, sun_ra, sun_dec, bin_time_mid)[0]

#calculate the earth data
earth_data = calc.earth_ang_bin(detector_name, day, bin_time_mid, data_type)
earth_ang_bin = earth_data[0]
#earth_ang_bin = calc.ang_eff(earth_ang_bin, echan)[0]
earth_ang_bin = calc.earth_occ_eff(earth_ang_bin, echan)

#read the SFL data
flares = rf.flares(year)
flares_day = flares[0]
flares_time = flares[1]
if np.any(flares_day == day) == True:
    flares_today = flares_time[:,np.where(flares_day == day)]
    flares_today = np.squeeze(flares_today, axis=(1,))/3600.
else:
    flares_today = np.array(-5)

#read the mcilwain parameter data
sat_time = rf.poshist(day)[0]
lat_data = rf.mcilwain(day)
mc_b = lat_data[1]
mc_l = lat_data[2]

mc_b = calc.intpol(mc_b, day, 0, sat_time, bin_time_mid)[0]
mc_l = calc.intpol(mc_l, day, 0, sat_time, bin_time_mid)[0]

magnetic = mc_l
magnetic = magnetic - np.mean(magnetic, dtype=np.float64)

#constant function corresponding to the diffuse y-ray background
cgb = np.ones(len(total_rate))

#counts[120000:] = 0
cgb[np.where(total_rate == 0)] = 0
earth_ang_bin[np.where(total_rate == 0)] = 0
sun_ang_bin[np.where(sun_occ == 0)] = 0
sun_ang_bin[np.where(total_rate == 0)] = 0
magnetic[np.where(total_rate == 0)] = 0

#remove vertical movement from scaling sun_ang_bin
sun_ang_bin[sun_ang_bin>0] = sun_ang_bin[sun_ang_bin>0] - np.min(sun_ang_bin[sun_ang_bin>0])

work_module.py:2542: RuntimeWarning: invalid value encountered in arccos
  ang_det_sun = np.arccos(scalar_product)
work_module.py:2102: RuntimeWarning: invalid value encountered in arccos
  ang_det_geo = np.arccos(scalar_product)


In [3]:
saa_exits = [0]
for i in range(1, len(total_rate)):
    if np.logical_and(total_rate[i-1] == 0, total_rate[i] != 0):
        print i
        saa_exits.append(i)
saa_exits = np.array(saa_exits)

if saa_exits[1] - saa_exits[0] < 10:
    saa_exits = np.delete(saa_exits, 0)

2399
22312
167308
185443
202562
219471
236937
255193
273998


In [4]:
#def exp_func(x, a, b, c):
#    return a*np.exp(-b*x) + c
def exp_func(x, a, b, i, addition):
#    if saa_exits[i] == 0:
#        addition = 0
#    elif saa_exits[i] < 200:
#        addition = 10
#    else:
#        addition = 9
    x_func = x[saa_exits[i]+math.fabs(addition):] - x[saa_exits[i]+math.fabs(addition)]
    func = math.fabs(a)*np.exp(-math.fabs(b)*x_func)
    zeros = np.zeros(len(x))
    zeros[saa_exits[i]+addition:] = func
    zeros[np.where(total_rate==0)] = 0
    return zeros

In [5]:
deaths = len(saa_exits)

exp = []
for i in range(0, deaths):
    exp = np.append(exp, [40., 0.001])

In [6]:
def fit_function(x, a, b, c, d, addition, exp1, exp2, deaths):
        this_is_it = a*cgb + b*magnetic + c*earth_ang_bin + d*sun_ang_bin
        for i in range(0, deaths):
            this_is_it = np.add(this_is_it, exp_func(x, exp1[i], exp2[i], i, addition))
        return this_is_it

def wrapper_fit_function(x, deaths, a, b, c, d, addition, *args):
    exp1 = args[::2]
    exp2 = args[1::2]
    return fit_function(x, a, b, c, d, addition, np.fabs(exp1), np.fabs(exp2), deaths)

x0 = np.append(np.array([20., -1., 5]), exp)
sigma = np.array((counts + 1)**(0.5))

In [7]:
energy_eff_width = np.array([0.138, 0.953, 1., 1.010, 0.859, 0.266, 0.109, 0.050])

cgb_ref = np.array([5.49, 2.69, 1., 0.443, 0.164, 0.0185, 0.00600, 0.00212]) * energy_eff_width
earth_occ_ref = np.array([6.994, 3.3976, 1., 0.23507, -0.061, -0.0586, -0.04544, -0.03653]) * energy_eff_width

#cgb_ref = np.array([0.3927, 1.7199, 1., 0.6736, 0.5674, 0.1110, 0.1557, 0.1767])
#earth_occ_ref = np.array([0.7620, 3.0449, 1., 0.2801, -0.1018, -0.0665, -0.1095, -0.1304])

cgb_e2 = 248.732651279 #n3: 243.029530211
earth_occ_e2 = -153.385764169 #n3: -176.022964251

cgb_coeff = cgb_e2*cgb_ref[echan]
earth_occ = earth_occ_e2*earth_occ_ref[echan]

In [8]:
fit_results = optimization.curve_fit(lambda x, b, d, addition, *args: wrapper_fit_function(x, deaths, cgb_coeff, b, earth_occ, d, addition, *args), bin_time_mid, counts, x0, sigma, maxfev = 100000)
coeff = fit_results[0]

print 'CGB coefficient:',cgb_coeff
print 'Magnetic field coefficient:',coeff[0]
print 'Earth angle coefficient:',earth_occ
print 'Sun angle coefficient:',coeff[1]
print 'Addition: ', coeff[2]
print 'starting exp:',coeff[3],'&',coeff[4]
print 'first SAA:',coeff[5],'&',coeff[6]

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


CGB coefficient: 637.643562839
Magnetic field coefficient: -84.7018250595
Earth angle coefficient: -496.649729141
Sun angle coefficient: -7.110087648
Addition:  5.0
starting exp: 0.414873216639 & 0.0289097817028
first SAA: 60.2398118941 & 0.00411206227894


In [9]:
b = coeff[0]
d = coeff[1]
addition = coeff[2]
exp1 = coeff[3::2]
exp2 = coeff[4::2]

fit_curve = fit_function(bin_time_mid, cgb_coeff, b, earth_occ, d, addition, exp1, exp2, deaths)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [10]:
#####plot-algorhythm#####
#convert the x-axis into hours of the day
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours


###plot each on the same axis as converted to counts###
fig, ax1 = plt.subplots()

plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Countrate')
plot2 = ax1.plot(plot_time_bin, fit_curve, 'r-', label = 'Fit')
plot3 = ax1.plot(plot_time_sat, d*sun_ang_bin, 'y-', label = 'Sun angle')
plot4 = ax1.plot(plot_time_sat, earth_occ*earth_ang_bin, 'c-', label = 'Earth angle')
plot5 = ax1.plot(plot_time_sat, b*magnetic, 'g-', label = 'Magnetic field')
plot6 = ax1.plot(plot_time_sat, cgb_coeff*cgb, 'b--', label = 'Cosmic y-ray background')
#plot7 = ax1.plot(plot_time_sat, j2000_orb, 'y--', label = 'J2000 orbit')
#plot8 = ax1.plot(plot_time_sat, geo_orb, 'g--', label = 'Geographical orbit')

#plot vertical lines for the solar flares of the day
if np.all(flares_today != -5):
    if len(flares_today[0]) > 1:
        for i in range(0, len(flares_today[0])):
            plt.axvline(x = flares_today[0,i], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
            plt.axvline(x = flares_today[1,i], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
    else:
        plt.axvline(x = flares_today[0], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
        plt.axvline(x = flares_today[1], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')

plots = plot1 + plot2 + plot3 + plot4 + plot5 + plot6
labels = [l.get_label() for l in plots]
ax1.legend(plots, labels, loc=1)

ax1.grid()

ax1.set_xlabel('Time of day in 24h')
ax1.set_ylabel('Countrate')

#ax1.set_xlim([9.84, 9.85])
ax1.set_xlim([-0.5, 24.5])
ax1.set_ylim([-500, 750])

plt.title(data_type + '-countrate-fit of the ' + detector_name + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year))


figure_path = '/home/tkili/Work/Fits/' + str(day) + '/'
if not os.access(figure_path, os.F_OK):
    os.mkdir(figure_path)
if counts.all == total_rate.all:
    figure_name = detector_name + 'ref_tot.png'
else:
    figure_name = detector_name + '_ref_e' + str(echan) + '.png'
            
fig = plt.gcf() # get current figure
fig.set_size_inches(20, 12)
            
figure = os.path.join(figure_path, figure_name)
plt.savefig(figure, bbox_inches='tight', dpi = 80)
            
#plt.show()
            
fig.clf()
plt.clf()

